In [ ]:
import pandas as pd
import numpy as np
import glob
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Must be a value betweeen 0-100
INPUT = 20

In [ ]:
df = pd.read_csv('data/sample.csv', index_col=0)

In [ ]:
def pareto_analysis_by_product(dataframe, percentile):
    """
    Returns a statement?? 
    """
    df = dataframe
    
    # total # of products for the month
    total_products = len(df.index)
        
    # sort dataframe in desc order by amount (price)
    df = df.sort_values(by='amount', ascending=False)

    # perc share amount spent. not neccessary for calc but good for ground truth
    total_amount = df.amount.sum()
    df['perc_share_amount'] = (df['amount'] / total_amount)*100
  
    # cumulative percentile
    df['cum_sum_amount'] = df['amount'].cumsum()
    df['cum_sum_perc_amount'] = 100*df['cum_sum_amount']/total_amount

    # filter the df where cum_sum_perc_amount reach 80.0
    df = df[df['cum_sum_perc_amount'] < percentile]

    # now calculate the KPI
    num_products = len(df.index)
    kpi = round(((num_products/total_products)*100), 2)
    
    return(f"{percentile}% of the amount spent was generated by {kpi}% of unique products.")


In [ ]:
pareto_analysis_by_product(df, INPUT)

# Make pareto table for chart
Code modified from: https://newbedev.com/how-to-overlay-two-plots-in-same-figure-in-plotly-create-pareto-chart-in-plotly

In [ ]:
def make_pareto_table_by_product(dataframe, percentile):
    """Returns a modified CSV of the original df 
    with cum sum and cum perc.
    
    Table saved to: data/plotly_data.csv'
    """
    
    df = dataframe
    
    # drop unnecessary column
    df = df.drop(columns='customer_id')
    
    # sort values before calculating cum sum & cum perc
    df = df.sort_values(by=['amount'], ascending=False)
    df['cumulative_sum'] = df.amount.cumsum()
    df['cumulative_perc'] = 100*df.cumulative_sum/df.amount.sum()
    
    # make two tables:
    # the first table: once cumulative_perc hits INPUT%
    # the second table: after cumulative_perc hits INPUT%
    top_perc = df.drop(df[df.cumulative_perc > percentile].index)
    bottom_perc = df.drop(df[df.cumulative_perc <= percentile].index)
    
    # sanity check. did we lose any rows? nope
    # print(len(top_80), len(bottom_20), len(df))
    
    # collapse bottom_20 into one row: 
    # product | amount
    # other   | 10000
    
    total_amount = bottom_perc['amount'].sum()
    bottom_perc = {'product': 'other', 'amount': total_amount}

    top_perc = top_perc[['product', 'amount']]

    temp_df = top_perc.append(bottom_perc, ignore_index = True)
    
    # now calculate cum sum & cum perc for new table
    temp_df['cumulative_sum'] = temp_df.amount.cumsum()
    temp_df['cumulative_perc'] = 100*temp_df.cumulative_sum/temp_df.amount.sum()
    
    # save to csv!
    temp_df.to_csv('data/plotly_data.csv', index=True)

make_pareto_table_by_product(df, INPUT)

# Make Plotly pareto chart!

In [ ]:
# read table 
temp_df = pd.read_csv('data/plotly_data.csv', index_col=0)

def pareto_chart(dataframe):
    """
    Reads the pareto table, returns a pareto fig.
    """
  
    # variables
    cat = 'product'
    num = 'amount'
    title = 'This is a Pareto chart in Plotly'
    
    trace1 = go.Bar(
    x=dataframe[cat],
    y=dataframe[num],
    name=num,
    marker=dict(
        color='rgb(34,163,192)'
               )
    )
    trace2 = go.Scatter(
    x=dataframe[cat],
    y=dataframe['cumulative_perc'],
    name='Cumulative Percentage',
    yaxis='y2'

    )

    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(trace1)
    fig.add_trace(trace2,secondary_y=True)
    fig['layout'].update(height = 600, width = 800, title = title, xaxis=dict(
      tickangle=-90
    ))

    return fig

update_pareto_chart(temp_df)

In [ ]:
import dash
from dash import dcc
from dash import html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=pareto_chart(temp_df))
])

app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter